In [103]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
import scipy.stats as stat

In [104]:
commune= pd.read_excel("commune.xlsx")
enroll= pd.read_csv("enroll.csv")
industry= pd.read_csv("industry.csv")
ord = pd.read_csv("ord.csv")
study_domain= pd.read_csv("study_domain.csv")
technology= pd.read_csv("technology.csv")
transaction= pd.read_csv("transaction.csv")
quest= pd.read_csv("quest.csv")

In [105]:
quest.dropna(inplace=True)

In [106]:
newquest=quest.drop(["modified_at","department","current_employed","Unnamed: 0","study_domain","university"],axis=1,inplace=False)

## create age column

In [107]:
newquest.drop(newquest[newquest["dob"]=="3 aout 1977"].index,inplace=True)

In [108]:
newquest["dob"]=newquest["dob"].apply(lambda x: x.replace("/",""))

In [109]:
newquest["dob"].unique()

array(['29091996', '17051996', '24111996', '20071997', '03121996',
       '30041988', '10091996', '03021991', '10071997', '20111994',
       '30121994', '09051992', '26031992', '01111994', '19011999',
       '09101987', '02051995', '05021992', '21091995', '05121995',
       '25121993', '09121998', '28022021', '26101991', '27081986',
       '13021977', '23081986', '13011989', '14042000', '31121998',
       '14051995', '04101996', '10011997', '22041998', '26061989',
       '16101996', '01031996', '11111994', '03091989', '14091996',
       '13121987', '13101996', '10122001', '01111996', '17041996',
       '31072000', '31051995', '09121997', '19101993', '14041986',
       '03081990', '05072002', '05051996', '22031992', '07091996',
       '28071994', '25061997', '30101995', '23081998', '17051997',
       '25121998', '01101995', '31031996', '20091993', '15101983',
       '09071995', '19021999', '20091998', '09041995', '28101998',
       '22061996', '01061996', '16111983', '15121995', '281219

In [110]:
newquest["dob"]=pd.to_datetime(newquest["dob"],format='%d%m%Y')

In [111]:
now = pd.Timestamp('now')
newquest['age'] = (now - newquest['dob']).astype('<m8[Y]')

In [112]:
newquest.drop("dob",axis=1,inplace=True)

## create Communename column

In [113]:
newquest.commune.unique()

array(['ht0111', 'ht0112', 'ht0521', 'ht0118', 'ht0113', 'ht0114',
       'ht0131', 'ht0434', 'ht0731', 'ht0211', 'ht0711', 'ht0117',
       'ht0122', 'ht0322', 'ht0121', 'ht0542', 'ht0721', 'ht0432',
       'ht0134', 'ht0116', 'ht0531', 'ht0133', 'ht0123', 'ht0421',
       'ht0532', 'ht0342', 'ht0311', 'ht0611', 'ht0115', 'ht0313',
       'ht0921', 'ht0621', 'ht0412', 'ht1012'], dtype=object)

In [114]:
commune.head()

,Commune_en,Commune_FR,Commune_Id,Departement,ADM1_PCODE
0,Abricots,Abricots,HT0812,Grande'Anse,HT08
1,Acul du Nord,Acul du Nord,HT0321,North,HT03
2,Anse-a-Foleur,Anse-à-Foleur,HT0922,North-West,HT09
3,Anse-a-Pitre,Anse-à-Pître,HT0234,South-East,HT02
4,Anse-a-Veau,Anse-à-Veau,HT1021,Nippes,HT10


In [115]:
commune.drop(["Commune_en","Departement","ADM1_PCODE"],axis=1,inplace=True)

In [116]:
commune.set_axis(["commune_name","commune"],axis=1,inplace=True)
commune["commune"]=commune["commune"].apply(lambda x : x.lower())

In [117]:
newquest.shape


(242, 11)

In [118]:
newquest=pd.merge(newquest,commune,on="commune")
newquest.drop("commune",axis=1,inplace=True)

## Create enroll date

In [119]:
newenroll=enroll[["created_at","quest_id","user_id"]]
newenroll.shape

(148, 3)

In [120]:
newquest=pd.merge(newenroll,newquest,on="quest_id",how='right')

In [121]:
newquest.rename(columns={"created_at_x":"enroll_date","created_at_y":"application_date","hear_AA_1":"communication_channel","after_AA":"bootcamp_interest"},inplace=True)

In [122]:
newquest.head()

,enroll_date,quest_id,user_id,gender,application_date,education_level,formal_sector_job,have_computer_home,internet_at_home,communication_channel,bootcamp_interest,age,commune_name
0,NaN,905ddcf2-ce95-11eb-9a92-7c67a234f601,NaN,female,2021-02-16T16:01:10.861844Z,Bachelors (bacc +4),unknown,Yes,Yes,Friend,Finding a job/internship,24.0,Port-au-Prince
1,2021-02-20T21:20:37.450Z,905e2608-ce95-11eb-809f-7c67a234f601,40117659.0,male,2021-02-16T16:56:30.579504Z,Bachelors (bacc +4),unknown,Yes,Yes,Friend,Improving my data analysis skills,25.0,Port-au-Prince
2,NaN,905e4cf5-ce95-11eb-96a9-7c67a234f601,NaN,male,2021-02-16T16:57:55.754541Z,Bachelors (bacc +4),unknown,Yes,Yes,ESIH,Improving my data analysis skills,24.0,Port-au-Prince
3,NaN,905e73fd-ce95-11eb-a7b0-7c67a234f601,NaN,male,2021-02-16T17:05:40.907948Z,High School (16 years),unknown,Yes,Yes,WhatsApp,Mentorship,24.0,Port-au-Prince
4,NaN,905e9b27-ce95-11eb-9c37-7c67a234f601,NaN,male,2021-02-16T18:35:14.430837Z,Other,unknown,Yes,Yes,ESIH,Improving my data analysis skills,24.0,Port-au-Prince


## Create is_enroll(Yes/No) column

In [123]:
newquest["is_enroll"]=newquest["enroll_date"].apply(lambda x: "No" if pd.isnull(x) else "Yes" )

## Payment Date

In [124]:
newtransac= transaction[["created_at","user_id"]]
newtransac.shape

(60, 2)

In [125]:
neword=ord[["quest_id","created_at"]]
neword.shape

(18, 2)

In [126]:
restran=pd.merge(newtransac,neword,on="created_at",how="outer")
restran.rename(columns={"created_at":"Payment date"},inplace=True)
restran


,Payment date,user_id,quest_id
0,2021-03-08T05:08:50.832Z,40976440.0,NaN
1,2021-03-08T17:26:35.841Z,41179271.0,NaN
2,2021-03-08T18:52:09.147Z,40973512.0,NaN
3,2021-03-08T19:06:32.837Z,41177453.0,NaN
4,2021-03-08T22:10:00.796Z,40797121.0,NaN
...,...,...,...
73,2021-02-18T14:45:02.292Z,NaN,905f5dc1-ce95-11eb-bfa7-7c67a234f601
74,2021-02-17T13:48:38.371Z,NaN,NaN
75,2021-02-17T11:08:54.385Z,NaN,9062b754-ce95-11eb-b31e-7c67a234f601
76,2021-02-16T20:09:31.627Z,NaN,9069789f-ce95-11eb-9788-7c67a234f601


In [127]:
newquest=pd.merge(newquest,restran,how='left',on=["quest_id"])

In [128]:
newquest["Payment date"].isnull().value_counts()

True     228
False     14
Name: Payment date, dtype: int64

## Create payed(Yes/No) column

In [129]:
newquest["payed"]=newquest["Payment date"].apply(lambda x: "No" if pd.isnull(x) else "Yes" )


## Create dummies list for technology

In [130]:
technology.head()

,Unnamed: 0,key,values,quest_id
0,0,item1,R,905ddcf2-ce95-11eb-9a92-7c67a234f601
1,1,item11,Excel,905ddcf2-ce95-11eb-9a92-7c67a234f601
2,2,item2,Python,905e2608-ce95-11eb-809f-7c67a234f601
3,3,item3,Java,905e2608-ce95-11eb-809f-7c67a234f601
4,4,item4,JavaScript,905e2608-ce95-11eb-809f-7c67a234f601


In [131]:
newtech=technology[["values","quest_id"]]

In [132]:
newtech=newtech.groupby("quest_id")['values'].apply(lambda x: ','.join(x.astype(str))).reset_index()

In [133]:
newtech.head(15)

,quest_id,values
0,905ddcf2-ce95-11eb-9a92-7c67a234f601,"R,Excel"
1,905e2608-ce95-11eb-809f-7c67a234f601,"Python,Java,JavaScript,SQL,Excel,Bash,Git"
2,905e4cf5-ce95-11eb-96a9-7c67a234f601,"Excel,Java,JavaScript,PHP,Python"
3,905e73fd-ce95-11eb-a7b0-7c67a234f601,"PHP,Bash"
4,905e9b27-ce95-11eb-9c37-7c67a234f601,"Python,Excel,JavaScript,Java,SQL"
5,905ec1ec-ce95-11eb-8136-7c67a234f601,"R,Python,Java,PHP,SQL,Excel"
6,905ec1ed-ce95-11eb-829c-7c67a234f601,"PHP,Excel,Java,JavaScript"
7,905ef688-ce95-11eb-9b57-7c67a234f601,Excel
8,905f0fbd-ce95-11eb-b153-7c67a234f601,"Excel,Git,Java,PowerBI or Tableau,other"
9,905f0fbe-ce95-11eb-8396-7c67a234f601,"Excel,JavaScript,Python"


In [134]:
newtech.shape

(244, 2)

In [135]:
dum=newtech["values"].str.get_dummies(sep=",")

In [136]:
newtech=newtech.join(dum)
newtech.drop("values",axis=1,inplace=True)

In [137]:
newtech.head()

,quest_id,Bash,Excel,Git,Java,JavaScript,PHP,PowerBI or Tableau,Python,R,SQL,VBA,other
0,905ddcf2-ce95-11eb-9a92-7c67a234f601,0,1,0,0,0,0,0,0,1,0,0,0
1,905e2608-ce95-11eb-809f-7c67a234f601,1,1,1,1,1,0,0,1,0,1,0,0
2,905e4cf5-ce95-11eb-96a9-7c67a234f601,0,1,0,1,1,1,0,1,0,0,0,0
3,905e73fd-ce95-11eb-a7b0-7c67a234f601,1,0,0,0,0,1,0,0,0,0,0,0
4,905e9b27-ce95-11eb-9c37-7c67a234f601,0,1,0,1,1,0,0,1,0,1,0,0


In [138]:
newquest.shape

(242, 17)

In [139]:
newquest=pd.merge(newquest,newtech,how='left',on="quest_id",validate="1:m")

## Create study domain dummy list

In [140]:
newstu=study_domain[["values","quest_id"]]

In [141]:
newstu=newstu.groupby("quest_id")['values'].apply(lambda x: ','.join(x.astype(str))).reset_index()

In [142]:
dum2=newstu["values"].str.get_dummies(sep=",")

In [143]:
newstu=newstu.join(dum2)
newstu.drop("values",axis=1,inplace=True)

In [144]:
newquest=pd.merge(newquest,newstu,how='left',on="quest_id",validate="1:1")

## End of transformation

In [145]:
newquest.columns

Index(['enroll_date', 'quest_id', 'user_id_x', 'gender', 'application_date',
       'education_level', 'formal_sector_job', 'have_computer_home',
       'internet_at_home', 'communication_channel', 'bootcamp_interest', 'age',
       'commune_name', 'is_enroll', 'Payment date', 'user_id_y', 'payed',
       'Bash', 'Excel', 'Git', 'Java', 'JavaScript', 'PHP',
       'PowerBI or Tableau', 'Python', 'R', 'SQL', 'VBA', 'other_x',
       'Accounting', 'Computer Science', 'Economics', 'Electrical Engineering',
       'Law', 'Management', 'Medicine', 'Statistics', 'other_y'],
      dtype='object')

In [146]:
newquest.shape

(242, 38)

In [147]:
newquest.rename(columns={"formal_sector_job":"Job is formal","have_computer_home":"Have computer at home","internet_at_home":"Have internet at home","other_y":"ST_other","other_x":"T_other"},inplace="True")

In [148]:
finalset=newquest[["quest_id","gender","age","commune_name","application_date","enroll_date","is_enroll","education_level","communication_channel","bootcamp_interest","Payment date","payed","Bash",
       "Excel", "Git", "Java", "JavaScript", "PHP", "PowerBI or Tableau",
       "Python", "R", "T_other","Accounting",
       "Computer Science", "Economics", "Electrical Engineering", "Law",
       "Management", "Medicine", "Statistics", "ST_other","Job is formal","Have computer at home","Have internet at home"]]

In [149]:
finalset.head()

,quest_id,gender,age,commune_name,application_date,enroll_date,is_enroll,education_level,communication_channel,bootcamp_interest,...,Economics,Electrical Engineering,Law,Management,Medicine,Statistics,ST_other,Job is formal,Have computer at home,Have internet at home
0,905ddcf2-ce95-11eb-9a92-7c67a234f601,female,24.0,Port-au-Prince,2021-02-16T16:01:10.861844Z,NaN,No,Bachelors (bacc +4),Friend,Finding a job/internship,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,unknown,Yes,Yes
1,905e2608-ce95-11eb-809f-7c67a234f601,male,25.0,Port-au-Prince,2021-02-16T16:56:30.579504Z,2021-02-20T21:20:37.450Z,Yes,Bachelors (bacc +4),Friend,Improving my data analysis skills,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,Yes,Yes
2,905e4cf5-ce95-11eb-96a9-7c67a234f601,male,24.0,Port-au-Prince,2021-02-16T16:57:55.754541Z,NaN,No,Bachelors (bacc +4),ESIH,Improving my data analysis skills,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,unknown,Yes,Yes
3,905e73fd-ce95-11eb-a7b0-7c67a234f601,male,24.0,Port-au-Prince,2021-02-16T17:05:40.907948Z,NaN,No,High School (16 years),WhatsApp,Mentorship,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,unknown,Yes,Yes
4,905e9b27-ce95-11eb-9c37-7c67a234f601,male,24.0,Port-au-Prince,2021-02-16T18:35:14.430837Z,NaN,No,Other,ESIH,Improving my data analysis skills,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,unknown,Yes,Yes


# Ayiti Analytics Probability and Statistics Project

Use all the files to have a final datasets to have the following columns
   <ul>
    <li>questid</li>
    <li>gender</li>
    <li>age (age of applicant)</li>
    <li>communeName (use data prep in commune file)</li>
    <li>application_date (created at in quest file)</li>
    <li>enroll_date (created_at in enroll file)</li>
    <li>is_enroll (Yes/No) (use data prep in enroll file)</li>
    <li>Education Level</li>
   <li>Communication channels(hear_AA1)</li>
    <li>Bootcamp Insterest (after_AA)</li>
    <li>Payement Date (use ord and transaction files)</li>
    <li>Payed (Yes/No)</li>
    <li>list Technologies as columns based (use get_dummies)</li>
    <li>list  Study domains (use get_dummies)</li>
    <li>Job is formal</li>
    <li>Have computer at home</li>
    <li>Have internet at home</li>
    
   </ul>
   

In [150]:
finalset.age.unique()

array([24., 25., 33., 30., 26., 29., 22., 27.,  0., 34., 44., 32., 21.,
       23., 31., 19., 20., 35., 37., 40., 28., 46., -1., 50., 39., 49.,
       38., 45.,  1., 41., 36.])

###  1. How many observations and variables are there in the dataset
 

In [151]:
finalset.shape

(242, 34)

### Answer-1 : Their is 242 observations and 34 variables in the data set

### 2.A Calculate the average age of the observations
   * On the whole dataset
   * On the whole male dataset
   * On all the female dataset
### 2.B Calculate 
   * variance of the whole datatset,male and female
   * kurtosis of the whole datatset,male and female
   * skewness of the whole datatset,male and female
### 2.C Does the age distribution follow a normal distribution 
   

In [152]:
aveall=finalset.age.mean()
avesex=finalset[["age","gender"]].groupby("gender").mean()
print(aveall,avesex)

26.760330578512395               age
gender           
female  24.909091
male    27.171717


### Answer-2A : The average age  is 27.17 years for males, 24.9 years for females and 26.76 for the overall dataset

In [153]:
varall=finalset.age.var()
varsex=finalset[["age","gender"]].groupby("gender").var()
print(varall,varsex)

51.45269366619796               age
gender           
female  53.665962
male    50.295237


### Answer-2B : The age variance is 50.29 years for males, 53.66 years for females and 51.45 for the overall dataset

In [154]:
kurall=finalset.age.kurtosis()
kurmale=finalset[["age","gender"]][finalset["gender"]=="male"]
kurfemale=finalset[["age","gender"]][finalset["gender"]=="female"]
print("alldata kurtosis : ",kurall)
print("Male kurtosis : ",kurmale.kurtosis())
print("female kurtosis : ",kurfemale.kurtosis())

alldata kurtosis :  5.19688723572512
Male kurtosis :  age    5.261487
dtype: float64
female kurtosis :  age    5.961896
dtype: float64


### Answer-2B : The age kurtosis is 5.26 for males distribution, 5.96  for females distribution and 5.19 for the overall dataset distribution

In [155]:
skeall=finalset.age.skew()
skemale=finalset[["age","gender"]][finalset["gender"]=="male"]
skefemale=finalset[["age","gender"]][finalset["gender"]=="female"]
print("alldata skewness : ",skeall)
print("Male skewness : ",skemale.skew())
print("female skewness : ",skefemale.skew())

alldata skewness :  -0.7501166186507338
Male skewness :  age   -0.670249
dtype: float64
female skewness :  age   -1.158519
dtype: float64


### Answer-2B : The age skewness is -0.67 for males distribution, -1.15  for females distribution and -0.75 for the overall dataset distribution

In [156]:
agedis=finalset.age
stat.shapiro(agedis)

ShapiroResult(statistic=0.8533609509468079, pvalue=2.1055028312749573e-14)

### Answer-2C Considering the p-value =2.1055028312749573e-14, we reject the null hypothesis.We have sufficient evidence to say that the sample data does not come from a normal distribution.

### 3.Display the age frequency disbrution
   * On the whole dataset
   * On the whole male dataset
   * On all the female dataset

In [157]:
sns.histplot(data=finalset,x=finalset.age,bins=40,hue="gender",kde=True)
plt.title("Distribution of age frequency by gender")
plt.xlabel("Age")
plt.ylabel("Number of applicants")
plt.figure(figsize=(10,10))
plt.show()

<Figure size 720x720 with 0 Axes>

In [158]:
sns.histplot(data=finalset,x=finalset.age,bins=40,kde=True)
plt.title("Distribution of age frequency on the whole dataset")
plt.xlabel("Age")
plt.ylabel("Number of applicants")
plt.figure(figsize=(10,10))
plt.show()

<Figure size 720x720 with 0 Axes>

### 4. Can we say that the average age (24 years old) of the observations can be considered as the average age of the population likely to participate in this bootcamp. Justify your answer

### Answer-4 : 
* Yes, the average age of the observation can be considered as a estimation of the population average age. Since the observation are a sample of the population created by a random sampling mathod, the average is applicable to the population

### 5. Calculate the average age of participants for each communication channel

In [159]:
finalset[["age","communication_channel"]].groupby("communication_channel").mean()

,age
communication_channel,
Ayiti Analytics Website,27.266667
Bootcamp Alumni,26.857143
ESIH,23.588235
Facebook,25.842105
Friend,26.969697
Instagram,25.200000
LinkedIn,30.642857
WhatsApp,26.828125
other,28.444444


### 6. Display an age boxplot for each communication channel

In [160]:
sns.boxplot(data=finalset,x="communication_channel",y="age")
plt.title("Boxplot of age by Communication channel")
plt.xlabel("Communication channels")
plt.ylabel("Age (years)")
plt.xticks(rotation=90)
plt.figure(figsize=(25,25))
plt.show()

<Figure size 1800x1800 with 0 Axes>

### 7 .Is there a significant age difference between these groups

In [161]:
friend=finalset[finalset.communication_channel=="Friend"][["age"]]
ESIH=finalset[finalset.communication_channel=="ESIH"][["age"]]
Whatsapp=finalset[finalset.communication_channel=="WhatsApp"][["age"]]
bootcamp=finalset[finalset.communication_channel=="Bootcamp Alumni"][["age"]]
Linked=finalset[finalset.communication_channel=="LinkedIn"][["age"]]
Ayiti=finalset[finalset.communication_channel=="Ayiti Analytics Website"][["age"]]
inta=finalset[finalset.communication_channel=="Instagram"][["age"]]
other=finalset[finalset.communication_channel=="other"][["age"]]

In [162]:
stat.f_oneway(friend,ESIH,Whatsapp,bootcamp,Linked,Ayiti,inta,other)

F_onewayResult(statistic=array([1.20110272]), pvalue=array([0.30384838]))

In [196]:
stat.kruskal(friend,ESIH,Whatsapp,bootcamp,Linked,Ayiti,inta,other)

KruskalResult(statistic=10.603115885478474, pvalue=0.1568922138615949)

### Answer-7 : 
* Their is no statistical difference of age between the different communication channels (pvalue=0.30384838)

### 8.Plan a BA strategy for each communication channel regarding the age group

### Answer-8 : 
* Considering that the difference in the mean age is not statisticaly significant, neither of the communication channels allow us to select participant from a specific age group.
So their is no need to prioritize a communication channel over the other in th selection process.

### 9.According to the observations what is the probability of being in each channel of communication knowing your are a woman

In [165]:
chasex=finalset[["communication_channel","gender"]].value_counts().to_frame()

In [166]:
chasex.reset_index(inplace=True)

In [167]:
chasex=pd.pivot_table(data=chasex,index="communication_channel",values=0,columns="gender")

In [168]:
chasex["comm/male_prob"]= (chasex.male/(chasex.male.sum()))*100
chasex["comm/female_prob"]= (chasex.female/(chasex.female.sum()))*100

In [169]:
chasex

gender,female,male,comm/male_prob,comm/female_prob
communication_channel,,,,
Ayiti Analytics Website,4,11,5.555556,9.090909
Bootcamp Alumni,8,6,3.030303,18.181818
ESIH,5,12,6.060606,11.363636
Facebook,4,34,17.171717,9.090909
Friend,10,56,28.282828,22.727273
Instagram,1,4,2.020202,2.272727
LinkedIn,1,13,6.565657,2.272727
WhatsApp,9,55,27.777778,20.454545
other,2,7,3.535354,4.545455


### Answer-9 : A woman has a probability of 9.09% to in Ayiti analytics channel, 18.18% to be in the Bootcamp channel, 11.36% to be in the ESIH channel, 9.09% to be in the facebook channel, 22.72% to be in the friend channel, 2.27% to be in the instagram channel, 2.27% to be in the linkedin channel, 20.45% to be in the whatsapp channel and 4.54.2% to be in the other channel.

### 10.According to the observations what is the probability of being in each channel of communication knowing your are a man

### Answer-10 : A man has a probability of 5.55% to in Ayiti analytics channel, 3.03% to be in the Bootcamp channel, 6.06% to be in the ESIH channel, 17.17% to be in the facebook channel, 28.28% to be in the friend channel, 2.02% to be in the instagram channel, 6.56% to be in the linkedin channel, 27.77% to be in the whatsapp channel and 3.53% to be in the other channel.

### 11. Deduce the probability of being a woman knowing each communication channel

In [170]:
gen=finalset.gender.value_counts().to_frame()
gen["prob"]=gen.gender/gen.gender.sum()
gen

,gender,prob
male,198,0.818182
female,44,0.181818


In [171]:
chasex.reset_index()

gender,communication_channel,female,male,comm/male_prob,comm/female_prob
0,Ayiti Analytics Website,4,11,5.555556,9.090909
1,Bootcamp Alumni,8,6,3.030303,18.181818
2,ESIH,5,12,6.060606,11.363636
3,Facebook,4,34,17.171717,9.090909
4,Friend,10,56,28.282828,22.727273
5,Instagram,1,4,2.020202,2.272727
6,LinkedIn,1,13,6.565657,2.272727
7,WhatsApp,9,55,27.777778,20.454545
8,other,2,7,3.535354,4.545455


In [172]:
cha=finalset.communication_channel.value_counts().to_frame()
cha["comm_prob"]=cha.communication_channel/cha.communication_channel.sum()
cha.reset_index(inplace=True)
cha.rename(columns={"index":"communication_channel","communication_channel":"count"},inplace=True)

In [173]:
chasex=pd.merge(chasex,cha,how="left",on="communication_channel")
chasex

,communication_channel,female,male,comm/male_prob,comm/female_prob,count,comm_prob
0,Ayiti Analytics Website,4,11,5.555556,9.090909,15,0.061983
1,Bootcamp Alumni,8,6,3.030303,18.181818,14,0.057851
2,ESIH,5,12,6.060606,11.363636,17,0.070248
3,Facebook,4,34,17.171717,9.090909,38,0.157025
4,Friend,10,56,28.282828,22.727273,66,0.272727
5,Instagram,1,4,2.020202,2.272727,5,0.020661
6,LinkedIn,1,13,6.565657,2.272727,14,0.057851
7,WhatsApp,9,55,27.777778,20.454545,64,0.264463
8,other,2,7,3.535354,4.545455,9,0.037190


In [174]:
chasex["male/comm_prob"]=(chasex["comm/male_prob"]*gen.iloc[0,1])/chasex["comm_prob"]
chasex["female/comm_prob"]=(chasex["comm/female_prob"]*gen.iloc[1,1])/chasex["comm_prob"]
chasex

,communication_channel,female,male,comm/male_prob,comm/female_prob,count,comm_prob,male/comm_prob,female/comm_prob
0,Ayiti Analytics Website,4,11,5.555556,9.090909,15,0.061983,73.333333,26.666667
1,Bootcamp Alumni,8,6,3.030303,18.181818,14,0.057851,42.857143,57.142857
2,ESIH,5,12,6.060606,11.363636,17,0.070248,70.588235,29.411765
3,Facebook,4,34,17.171717,9.090909,38,0.157025,89.473684,10.526316
4,Friend,10,56,28.282828,22.727273,66,0.272727,84.848485,15.151515
5,Instagram,1,4,2.020202,2.272727,5,0.020661,80.000000,20.000000
6,LinkedIn,1,13,6.565657,2.272727,14,0.057851,92.857143,7.142857
7,WhatsApp,9,55,27.777778,20.454545,64,0.264463,85.937500,14.062500
8,other,2,7,3.535354,4.545455,9,0.037190,77.777778,22.222222


### Answer-11 :
* Their is a probability 26.66% of being a woman selected throught the Ayiti analytics channel
* Their is a probability 57.14% of being a woman selected throught the bootcamp alumni  channel
* Their is a probability 29.41% of being a woman selected throught the ESIH  channel
* Their is a probability 10.52% of being a woman selected throught the facebook  channel
* Their is a probability 15.15% of being a woman selected throught the friend  channel
* Their is a probability 20% of being a woman selected throught the Instagram  channel
* Their is a probability 7.14% of being a woman selected throught the Linkedin  channel
* Their is a probability 14.06% of being a woman selected throught the WhasApp  channel
* Their is a probability 22.22% of being a woman selected throught a other  channel
 

### 13. Deduce the probability of being a woman knowing each communication channel

In [175]:
# here your codes

### 14. Deduce the probability of being a man knowing each communication channel

### Answer-11 :  
* Their is a probability 73.33% of being a man selected throught the Ayiti analytics channel
* Their is a probability 42.85% of being a man selected throught the bootcamp alumni  channel
* Their is a probability 70.58% of being a man selected throught the ESIH  channel
* Their is a probability 89.47% of being a man selected throught the facebook  channel
* Their is a probability 84.84% of being a man selected throught the friend  channel
* Their is a probability 80% of being a man selected throught the Instagram  channel
* Their is a probability 92.85% of being a man selected throught the Linkedin  channel
* Their is a probability 85.93% of being a man selected throught the WhasApp  channel
* Their is a probability 77.77% of being a man selected throught a other  channel
 

### 15 Display a plot to see Gender vs Communication Channels .Is there any dependency between communication channels and gender?

In [176]:
chasex=chasex.reset_index()

In [177]:
fig,ax=plt.subplots(1,2)
ax[0].bar(data=chasex,x="communication_channel",height="female")
ax[1].bar(data=chasex,x="communication_channel",height="male")
ax[0].set_title("Females")
ax[1].set_title("Males")
plt.figure(figsize=(40,40))
fig.suptitle("Barplot of the communication channel by sex")
ax[0].set_xticklabels(["Ayiti Analytics Website","Bootcamp Alumni","ESIH","Facebook","Friend","Instagram","LinkedIn","WhatsApp","other"],rotation=90)
ax[1].set_xticklabels(["Ayiti Analytics Website","Bootcamp Alumni","ESIH","Facebook","Friend","Instagram","LinkedIn","WhatsApp","other"],rotation=90)
plt.show()

<ipython-input-177-a366bf873d89>:8: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax[0].set_xticklabels(["Ayiti Analytics Website","Bootcamp Alumni","ESIH","Facebook","Friend","Instagram","LinkedIn","WhatsApp","other"],rotation=90)
<ipython-input-177-a366bf873d89>:9: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax[1].set_xticklabels(["Ayiti Analytics Website","Bootcamp Alumni","ESIH","Facebook","Friend","Instagram","LinkedIn","WhatsApp","other"],rotation=90)


<Figure size 2880x2880 with 0 Axes>

In [178]:
value=np.array([chasex.female.values,chasex.male.values])
stat.chi2_contingency(value)[0:3]

(20.344036775285165, 0.009109933364814957, 8)

### Answer-15 : Considering that the p-value is 0.009109933364814957, we can reject the null hypothesis,so the gender and the communication channel are dependent of each other.

### 16 Use the same method to display plot and know if is there any dependency between communication channels and Bootcamp Insterest?

In [179]:
genboot=pd.crosstab(finalset.bootcamp_interest,finalset.gender)
genboot.reset_index(inplace=True)

In [180]:
fig,ax=plt.subplots(1,2)
ax[0].bar(data=genboot,x="bootcamp_interest",height="female")
ax[1].bar(data=genboot,x="bootcamp_interest",height="male")
ax[0].set_title("Females")
ax[1].set_title("Males")
plt.figure(figsize=(40,40))
fig.suptitle("Barplot of the bootcamp_interest  by gender")
ax[0].set_xticklabels(["Finding a job/internship","Improving my data analysis skills","Mentorship","Switching careers","Training to start my own company","other","unknown"],rotation=90)
ax[1].set_xticklabels(["Finding a job/internship","Improving my data analysis skills","Mentorship","Switching careers","Training to start my own company","other","unknown"],rotation=90)
plt.show()

<ipython-input-180-118988d3b45f>:8: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax[0].set_xticklabels(["Finding a job/internship","Improving my data analysis skills","Mentorship","Switching careers","Training to start my own company","other","unknown"],rotation=90)
<ipython-input-180-118988d3b45f>:9: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax[1].set_xticklabels(["Finding a job/internship","Improving my data analysis skills","Mentorship","Switching careers","Training to start my own company","other","unknown"],rotation=90)


<Figure size 2880x2880 with 0 Axes>

In [181]:
value=np.array([genboot.female.values,genboot.male.values])
stat.chi2_contingency(value)[0:3]

(1.5753991714929216, 0.9543296676146303, 6)

### Answer-16 : Considering that the p-value is 0.9543296676146303, we cannot reject the null hypothesis,so the gender and the bootcamp_interest are independent of each other.

### 17.Plan a BA  strategy for each communication channel regarding the gender 

### Answer-17 : 
* As the gender and communication channel are correlated we can increase our chance of having participant of a specific gender by prioritizing one channel over the other.
* We are more likely to select a female by prioritizing those channel: "Friend","whatsapp","Bootcamp Alumni"
* We are more likely to select a male by prioritizing those channel: "Friend","whatsapp","Facebook"
* As we can see the male and female most used communication channel diverge by one channel, therefore prioritizing facebook or Bootcamp alumni will influence tho poll of participant toward man or woman.


### 18.Plan a BA  strategy for each Bootcamp Insterest regarding the gender

### Answer-18 : 
* As the gender and bootcamp interest aren't correlated we can't increase our chance of having participant of a specific gender by prioritizing one interest  over the other.

### 19. Calculate 
    * P(Bootcamp Insterest,Communication channels,Gender/Payed=yes)
    * P(Bootcamp Insterest,Communication channels,Gender/Payed=no)

In [184]:
genpay=pd.crosstab(finalset.gender,finalset.payed)
genpay["gender/Yes_prob"]=(genpay.Yes/genpay.Yes.sum())*100
genpay["gender/No_prob"]=(genpay.No/genpay.No.sum())*100
genpay

payed,No,Yes,gender/Yes_prob,gender/No_prob
gender,,,,
female,39,5,35.714286,17.105263
male,189,9,64.285714,82.894737


### Answer-19 : 
* Their is a probability 35.71% of being a female knowing you have payed
* Their is a probability 64.28% of being a male knowing you have payed
* Their is a probability 82.89% of being a male knowing you haven't payed
* Their is a probability 17.10% of being a female knowing you haven't payed



In [185]:
compay=pd.crosstab(finalset.communication_channel,finalset.payed)
compay["com/Yes_prob"]=(compay.Yes/compay.Yes.sum())*100
compay["com/No_prob"]=(compay.No/compay.No.sum())*100
compay

payed,No,Yes,com/Yes_prob,com/No_prob
communication_channel,,,,
Ayiti Analytics Website,15,0,0.000000,6.578947
Bootcamp Alumni,13,1,7.142857,5.701754
ESIH,17,0,0.000000,7.456140
Facebook,36,2,14.285714,15.789474
Friend,61,5,35.714286,26.754386
Instagram,4,1,7.142857,1.754386
LinkedIn,14,0,0.000000,6.140351
WhatsApp,60,4,28.571429,26.315789
other,8,1,7.142857,3.508772


### Answer-19 : 
* Their is a probability 0.00% of using the Ayiti Analytics Website channel knowing you have payed
* Their is a probability 7.14% of using the Bootcamp Alumni channel knowing you have payed
* Their is a probability 0.00% of using the ESIH channel knowing you have payed
* Their is a probability 14.28% of using the Facebook channel knowing you have payed
* Their is a probability 35.71% of using the Friend channel knowing you have payed
* Their is a probability 7.14% of using the Instagram knowing you have payed
* Their is a probability 0.00% of using the LinkedIn knowing you have payed
* Their is a probability 28.57% of using the WhatsApp knowing you have payed
* Their is a probability 7.14% of using the other knowing you have payed


### Answer-19 : 
* Their is a probability 6.57% of using the Ayiti Analytics Website channel knowing you haven't payed
* Their is a probability 5.70% of using the Bootcamp Alumni channel knowing you haven't payed
* Their is a probability 7.45% of using the ESIH channel knowing you haven't payed
* Their is a probability 15.78% of using the Facebook channel knowing you haven't payed
* Their is a probability 26.75% of using the Friend channel knowing you haven't payed
* Their is a probability 1.75% of using the Instagram knowing you haven't payed
* Their is a probability 6.14% of using the LinkedIn knowing you haven't payed
* Their is a probability 26.31% of using the WhatsApp knowing you haven't payed
* Their is a probability 3.50% of using the other knowing you haven't payed


In [186]:
boopay=pd.crosstab(finalset.bootcamp_interest,finalset.payed)
boopay["boo/Yes_prob"]=(boopay.Yes/boopay.Yes.sum())*100
boopay["boo/No_prob"]=(boopay.No/boopay.No.sum())*100
boopay

payed,No,Yes,boo/Yes_prob,boo/No_prob
bootcamp_interest,,,,
Finding a job/internship,39,3,21.428571,17.105263
Improving my data analysis skills,118,10,71.428571,51.754386
Mentorship,12,0,0.000000,5.263158
Switching careers,20,0,0.000000,8.771930
Training to start my own company,36,1,7.142857,15.789474
other,2,0,0.000000,0.877193
unknown,1,0,0.000000,0.438596


### Answer-19 : 
* Their is a probability 21.42% of being interested by Finding a job/internship knowing you have payed
* Their is a probability 71.42% of being interested by Improving my data analysis skills knowing you have payed
* Their is a probability 0.00% of being interested by Mentorship  knowing you have payed
* Their is a probability 0.00% of being interested by Switching careers knowing you have payed
* Their is a probability 7.14% of being interested by Training to start my own company knowing you have payed
* Their is a probability 0.00% of being interested by other resons knowing you have payed
* Their is a probability 0.00% of not knowing the reasons knowing you have payed



### Answer-19 : 
* Their is a probability 17.10% of being interested by Finding a job/internship knowing you haven't payed
* Their is a probability 51.75% of being interested by Improving my data analysis skills knowing you haven't payed
* Their is a probability 5.26% of being interested by Mentorship  knowing you haven't payed
* Their is a probability 8.77% of being interested by Switching careers knowing you haven't payed
* Their is a probability 15.78% of being interested by Training to start my own company knowing you haven't payed
* Their is a probability 0.87% of being interested by other resons knowing you haven't payed
* Their is a probability 0.43% of not knowing the reasons knowing you haven't payed



### 20 reduce 
* P(Payed="yes"/Bootcamp Insterest="Increase Skill",Communication channels="Friend",Gender="male")
* P(Payed="no"/Bootcamp Insterest="Increase Skill",Communication channels="Friend",Gender="male")

* P(Payed="yes/no"/Gender="male")

In [187]:
gen.prob=gen.prob*100
gen

,gender,prob
male,198,81.818182
female,44,18.181818


In [188]:
pay=finalset.payed.value_counts().to_frame()
pay["prob"]=(pay.payed/pay.payed.sum())*100
pay

,payed,prob
No,228,94.214876
Yes,14,5.785124


In [189]:
genpay["Yes/male_prob"]=(genpay["gender/Yes_prob"]*pay.iloc[1,1])/gen.iloc[0,1]
genpay["No/male_prob"]=(genpay["gender/No_prob"]*pay.iloc[0,1])/gen.iloc[0,1]
genpay

payed,No,Yes,gender/Yes_prob,gender/No_prob,Yes/male_prob,No/male_prob
gender,,,,,,
female,39,5,35.714286,17.105263,2.525253,19.696970
male,189,9,64.285714,82.894737,4.545455,95.454545


### Answer-20
* The probability of paying the course while you are a man is 4.54%
* The probability of not paying the course while you are a man is 95.45%

* P(Payed="yes/no"/Bootcamp Insterest="Increase Skill")

In [190]:
boo=finalset.bootcamp_interest.value_counts().to_frame()
boo["prob"]=(boo.bootcamp_interest/boo.bootcamp_interest.sum())*100
boo

,bootcamp_interest,prob
Improving my data analysis skills,128,52.892562
Finding a job/internship,42,17.355372
Training to start my own company,37,15.289256
Switching careers,20,8.264463
Mentorship,12,4.958678
other,2,0.826446
unknown,1,0.413223


In [191]:
boopay["Yes/Impskil_prob"]=(boopay["boo/Yes_prob"]*pay.iloc[1,1])/boo.iloc[0,1]
boopay["No/Impskil_prob"]=(boopay["boo/No_prob"]*pay.iloc[0,1])/boo.iloc[0,1]
boopay

payed,No,Yes,boo/Yes_prob,boo/No_prob,Yes/Impskil_prob,No/Impskil_prob
bootcamp_interest,,,,,,
Finding a job/internship,39,3,21.428571,17.105263,2.34375,30.46875
Improving my data analysis skills,118,10,71.428571,51.754386,7.81250,92.18750
Mentorship,12,0,0.000000,5.263158,0.00000,9.37500
Switching careers,20,0,0.000000,8.771930,0.00000,15.62500
Training to start my own company,36,1,7.142857,15.789474,0.78125,28.12500
other,2,0,0.000000,0.877193,0.00000,1.56250
unknown,1,0,0.000000,0.438596,0.00000,0.78125


### Answer-20
* The probability of paying the course while you are interested in improving your skill is 7.81%
* The probability of not paying the course while you are  interested in improving your skill is 92.18%

* P(Payed="yes/no"/Communication channels="Friend")

In [192]:
com=finalset.communication_channel.value_counts().to_frame()
com["prob"]=(com.communication_channel/com.communication_channel.sum())*100
com

,communication_channel,prob
Friend,66,27.272727
WhatsApp,64,26.446281
Facebook,38,15.702479
ESIH,17,7.024793
Ayiti Analytics Website,15,6.198347
Bootcamp Alumni,14,5.785124
LinkedIn,14,5.785124
other,9,3.719008
Instagram,5,2.066116


In [193]:
compay["Yes/friend"]=(compay["com/Yes_prob"]*pay.iloc[1,1])/com.iloc[0,1]
compay["No/friend"]=(compay["com/No_prob"]*pay.iloc[0,1])/com.iloc[0,1]
compay

payed,No,Yes,com/Yes_prob,com/No_prob,Yes/friend,No/friend
communication_channel,,,,,,
Ayiti Analytics Website,15,0,0.000000,6.578947,0.000000,22.727273
Bootcamp Alumni,13,1,7.142857,5.701754,1.515152,19.696970
ESIH,17,0,0.000000,7.456140,0.000000,25.757576
Facebook,36,2,14.285714,15.789474,3.030303,54.545455
Friend,61,5,35.714286,26.754386,7.575758,92.424242
Instagram,4,1,7.142857,1.754386,1.515152,6.060606
LinkedIn,14,0,0.000000,6.140351,0.000000,21.212121
WhatsApp,60,4,28.571429,26.315789,6.060606,90.909091
other,8,1,7.142857,3.508772,1.515152,12.121212


### Answer-20
* The probability of paying the course while you used the friend channel is 7.57%
* The probability of not paying the course while you used the friend channel is 92.42%

### Based on these findings, propose strategies to increase our sales?

### Considering the finding of the 20th question, a increase in the course sale can be obtained by :
* Using adds targeting less man as they have a 95,45% probability of not buy the course
* Prioritizing communication channels other than the friend channel as those have a 92.42% probability of not buy the course
* Using adds not targeting people looking to improve their analytical skills as they have a 92.18% probability of not buy the course